# Loading Dependencies
---

In [ ]:
import sys

if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/gdrive')

    !pip install yfinance
    !pip install pdfplumber
    !rm -rf github
    !mkdir github
    !cd github && git clone --branch master https://github.com/dr315/Finances.git
    sys.path.append('/content/github/Finances')
    
    root = '/content/gdrive/MyDrive/'
else:
    print(sys.platform)
    if sys.platform == 'linux':
        root = '/home/doreis/Documents/'
    else:
        root = 'd:/'

import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import matplotlib.pyplot as plt
import ipywidgets as widgets
from Wallet import *

root += 'Investing/'
wallet = Wallet(root)
wallet.run(market='br')

# Realized Profit


In [ ]:
wallet.realized_profit_pivot_all

In [ ]:
wallet.realized_profit_pivot_fii

In [ ]:
wallet.realized_profit_pivot_stock

In [ ]:
wallet.realized_profit_df

In [ ]:
wallet.realized_profit_by_symbol_df

# Portifolio


In [ ]:
wallet.portifolio_df

# Performance Blueprint

In [ ]:
wallet.blueprint_df

# Dividends by month

In [ ]:
wallet.div_table

In [ ]:
wallet.pvt_div_table

# Taxation


## FII

### Swing Trade

### Day Trade

## Stocks


### Swing Trade

### Day Trade

# Plots

## Historical profitability

In [ ]:
import ipywidgets as widgets
periodPicker = widgets.Dropdown(options=['1 year', '2 years', 'All'], value='1 year')
periodPicker

In [ ]:
wallet.generate_charts()